# Setup and Import

In [1]:
# 📦 Imports
import pandas as pd
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from functools import reduce

In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime

# === Step 1: Load the saved schedule HTML ===
html_path = r"C:\Users\karan\Downloads\Premier League Scores & Fixtures _ FBref.com.html"
with open(html_path, "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

# === Step 2: Loop through each row and extract match info ===
data = []

for row in soup.find_all("tr"):
    # Gameweek is stored in <th data-stat="gameweek">
    gw_cell = row.find("th", {"data-stat": "gameweek"})
    if not gw_cell or gw_cell.get("scope") != "row":
        continue

    gameweek = gw_cell.text.strip()
    
    # Home and Away teams
    home_team_cell = row.find("td", {"data-stat": "home_team"})
    away_team_cell = row.find("td", {"data-stat": "away_team"})
    if not home_team_cell or not away_team_cell:
        continue

    home_team = home_team_cell.text.strip()
    away_team = away_team_cell.text.strip()

    # Match URL
    match_report_link = row.find("a", string="Match Report")
    if not match_report_link:
        continue

    match_url = "https://fbref.com" + match_report_link["href"]

    # Extract date from URL
    def extract_date_from_url(url):
        match = re.search(r'-(\w+)-(\d{1,2})-(\d{4})-Premier-League', url)
        if match:
            month_str, day, year = match.groups()
            try:
                return datetime.strptime(f"{month_str} {day} {year}", "%B %d %Y")
            except:
                return None
        return None

    match_date = extract_date_from_url(match_url)

    # Store all info
    data.append({
        "Home Team": home_team,
        "Away Team": away_team,
        "Gameweek": int(gameweek),
        "Match Date": match_date,
        "Match URL": match_url
    })

# === Step 3: Build and sort the DataFrame ===
df_matches = pd.DataFrame(data)
df_matches = df_matches.dropna(subset=["Match Date"]).sort_values("Match Date").reset_index(drop=True)

# === Step 4: Save to CSV or Excel ===
output_path = r"C:\Users\karan\Downloads\premier_league_match_info.csv"
df_matches.to_csv(output_path, index=False)

# Optional: Display first 10 rows
from IPython.display import display
display(df_matches.head(20))

,Home Team,Away Team,Gameweek,Match Date,Match URL
0,Manchester Utd,Fulham,1,2024-08-16,https://fbref.comhttps://fbref.com/en/matches/...
1,Ipswich Town,Liverpool,1,2024-08-17,https://fbref.comhttps://fbref.com/en/matches/...
2,Newcastle Utd,Southampton,1,2024-08-17,https://fbref.comhttps://fbref.com/en/matches/...
3,Everton,Brighton,1,2024-08-17,https://fbref.comhttps://fbref.com/en/matches/...
4,Nott'ham Forest,Bournemouth,1,2024-08-17,https://fbref.comhttps://fbref.com/en/matches/...
5,Arsenal,Wolves,1,2024-08-17,https://fbref.comhttps://fbref.com/en/matches/...
6,West Ham,Aston Villa,1,2024-08-17,https://fbref.comhttps://fbref.com/en/matches/...
7,Brentford,Crystal Palace,1,2024-08-18,https://fbref.comhttps://fbref.com/en/matches/...
8,Chelsea,Manchester City,1,2024-08-18,https://fbref.comhttps://fbref.com/en/matches/...
9,Leicester City,Tottenham,1,2024-08-19,https://fbref.comhttps://fbref.com/en/matches/...


In [4]:
import pandas as pd
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# === Step 1: Load match links CSV ===
csv_path = r"C:\Users\karan\Downloads\premier_league_match_info.csv"
df = pd.read_csv(csv_path)
df["Match URL"] = df["Match URL"].str.replace("https://fbref.comhttps://fbref.com", "https://fbref.com")

# Filter for Gameweek 1 matches
gw1_matches = df[df["Gameweek"] == 2].reset_index(drop=True)

# === Step 2: Set up Selenium headless browser ===
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

results = []

# === Step 3: Loop through Gameweek 1 matches ===
for i, row in gw1_matches.iterrows():
    url = row["Match URL"]
    home_team = row["Home Team"]
    away_team = row["Away Team"]
    print(f"Fetching (rendering): {url}")
    
    try:
        driver.get(url)
        time.sleep(2)  # Wait for JS to load
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Look under #content for all stat divs
        content_div = soup.find("div", {"id": "content"})

        if not content_div:
            print(f"❌ No content found in: {url}")
            continue

        # Extract both outfield and keeper divs
        for div in content_div.find_all("div", id=True):
            id_val = div["id"]

            # Match div_stats_<id>_summary or div_keeper_stats_<id>
            match = re.match(r"(div_stats|div_keeper_stats)_(.+?)(_summary)?$", id_val)
            if match:
                div_type = "summary" if "summary" in id_val else "keeper"
                team_id = match.group(2)
                
                # Assign based on appearance order: 1st = Home, 2nd = Away
                team_name = home_team if len([r for r in results if r["Match URL"] == url]) % 2 == 0 else away_team

                results.append({
                    "Match URL": url,
                    "Team Name": team_name,
                    "Div ID": team_id,
                    "Div Type": div_type
                })

    except Exception as e:
        print(f"❌ Failed to process {url}: {e}")

    time.sleep(1)

driver.quit()

# === Step 4: Save results ===
df_ids = pd.DataFrame(results).drop_duplicates().reset_index(drop=True)

# === Step 5: Show preview ===
from IPython.display import display
display(df_ids.head(100))

Fetching (rendering): https://fbref.com/en/matches/4692171a/Aston-Villa-Arsenal-August-24-2024-Premier-League
Fetching (rendering): https://fbref.com/en/matches/540cfb68/Crystal-Palace-West-Ham-United-August-24-2024-Premier-League
Fetching (rendering): https://fbref.com/en/matches/4d0079fb/Fulham-Leicester-City-August-24-2024-Premier-League
Fetching (rendering): https://fbref.com/en/matches/a24b7a43/Manchester-City-Ipswich-Town-August-24-2024-Premier-League
Fetching (rendering): https://fbref.com/en/matches/a641f3a0/Southampton-Nottingham-Forest-August-24-2024-Premier-League
Fetching (rendering): https://fbref.com/en/matches/fc8ab8b2/Brighton-and-Hove-Albion-Manchester-United-August-24-2024-Premier-League
Fetching (rendering): https://fbref.com/en/matches/1eef1717/Tottenham-Hotspur-Everton-August-24-2024-Premier-League
Fetching (rendering): https://fbref.com/en/matches/e76c15c9/Wolverhampton-Wanderers-Chelsea-August-25-2024-Premier-League
Fetching (rendering): https://fbref.com/en/matc

,Match URL,Team Name,Div ID,Div Type
0,https://fbref.com/en/matches/4692171a/Aston-Vi...,Aston Villa,8602292d,summary
1,https://fbref.com/en/matches/4692171a/Aston-Vi...,Arsenal,8602292d_passing,keeper
2,https://fbref.com/en/matches/4692171a/Aston-Vi...,Aston Villa,8602292d_passing_types,keeper
3,https://fbref.com/en/matches/4692171a/Aston-Vi...,Arsenal,8602292d_defense,keeper
4,https://fbref.com/en/matches/4692171a/Aston-Vi...,Aston Villa,8602292d_possession,keeper
...,...,...,...,...
95,https://fbref.com/en/matches/1eef1717/Tottenha...,Everton,d3fd31cc_possession,keeper
96,https://fbref.com/en/matches/1eef1717/Tottenha...,Tottenham,d3fd31cc_misc,keeper
97,https://fbref.com/en/matches/1eef1717/Tottenha...,Everton,d3fd31cc,keeper
98,https://fbref.com/en/matches/e76c15c9/Wolverha...,Wolves,8cec06e1,summary


In [5]:
import pandas as pd

# Assuming df_ids is already in memory and looks like:
# columns = ["Match URL", "Team Name", "Div ID", "Div Type"]

# Step 1: Only keep summary divs (they uniquely identify the team)
df_summary = df_ids[df_ids["Div Type"] == "summary"].copy()

# Step 2: Format div ID as required (e.g., div_stats_fd962109)
df_summary["Div Identifier"] = "div_stats_" + df_summary["Div ID"]

# Step 3: Create final table with 1 entry per team
team_div_map = df_summary[["Team Name", "Div Identifier"]].drop_duplicates().reset_index(drop=True)

# Step 4: Display
from IPython.display import display
display(team_div_map)

# Step 5: Save CSV and display download link
csv_path = "team_div_summary_map_gw1.csv"
team_div_map.to_csv(csv_path, index=False)

,Team Name,Div Identifier
0,Aston Villa,div_stats_8602292d
1,Arsenal,div_stats_18bb7c10
2,Crystal Palace,div_stats_47c64c55
3,West Ham,div_stats_7c21e445
4,Fulham,div_stats_fd962109
5,Leicester City,div_stats_a2d435b3
6,Manchester City,div_stats_b8fd03ef
7,Ipswich Town,div_stats_b74092de
8,Southampton,div_stats_33c895d4
9,Nott'ham Forest,div_stats_e4a775cb


In [6]:
import pandas as pd
import os

# Assuming df_ids is already in memory
df_summary = df_ids[df_ids["Div Type"] == "summary"].copy()
df_summary["Div Identifier"] = "div_stats_" + df_summary["Div ID"]
team_div_map = df_summary[["Team Name", "Div Identifier"]].drop_duplicates().reset_index(drop=True)

# Save directly to Downloads
downloads_path = os.path.join(os.path.expanduser("~"), "Downloads", "team_div_summary_map.csv")
team_div_map.to_csv(downloads_path, index=False)

print(f"✅ File saved to: {downloads_path}")

✅ File saved to: C:\Users\karan\Downloads\team_div_summary_map.csv


In [10]:
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from functools import reduce
import os

# Load data
match_df = pd.read_csv(os.path.expanduser("~/Downloads/premier_league_match_info.csv"))
team_div_df = pd.read_csv(os.path.expanduser("~/Downloads/team_div_summary_map.csv"))

# Fix malformed URLs if needed
match_df["Match URL"] = match_df["Match URL"].str.replace("https://fbref.comhttps://fbref.com", "https://fbref.com")

# Map team names to div base IDs (e.g., 'Manchester United': 'div_stats_fd96...')
team_to_divid = dict(zip(team_div_df["Team Name"], team_div_df["Div Identifier"]))

# Setup headless Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

# Process only first 10 games
#match_df = match_df.head(20)

final_rows = []

# === STAT TYPES ===
stat_types = ["summary", "passing", "passing_types", "defense", "possession", "misc"]

for _, row in match_df.iterrows():
    url = row["Match URL"]
    home_team = row["Home Team"]
    away_team = row["Away Team"]

    print(f"🔄 Processing: {home_team} vs {away_team}")
    
    try:
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        for team, opponent, side in [(home_team, away_team, "Home"), (away_team, home_team, "Away")]:
            div_base = team_to_divid.get(team)
            if not div_base:
                print(f"⚠️ No div ID for {team}")
                continue

            team_tables = {}

            for stat in stat_types:
                full_div_id = f"{div_base}_{stat}"
                div = soup.find("div", {"id": full_div_id})
                if not div:
                    continue

                table = div.find("table")
                if not table:
                    continue

                # Parse HTML table
                try:
                    header_rows = table.find("thead").find_all("tr")
                    headers = [th.text.strip() for th in header_rows[-1].find_all("th")]
                    rows = []
                    for row in table.find("tbody").find_all("tr"):
                        if 'class' in row.attrs and 'thead' in row['class']:
                            continue
                        player = row.find("th").text.strip()
                        stats = [td.text.strip() for td in row.find_all("td")]
                        rows.append([player] + stats)

                    df = pd.DataFrame(rows, columns=[headers[0]] + headers[1:])
                    if stat == "summary":
                        df["Home/Away"] = side
                        df["Team"] = team
                        df["Opponent"] = opponent
                    team_tables[stat.capitalize()] = df

                except Exception as e:
                    print(f"❌ Failed to parse table {full_div_id} for {team}: {e}")
                    continue

            # === Merge all tables into one per team ===
            dfs = []
            for label, df in team_tables.items():
                temp_df = df.copy()
                stat_prefix = label.replace(" ", "_")
                temp_df = temp_df.rename(columns={col: f"{stat_prefix}__{col}" for col in temp_df.columns if col != df.columns[0]})
                temp_df = temp_df.rename(columns={df.columns[0]: "Player"})
                dfs.append(temp_df)

            if dfs:
                merged_df = reduce(lambda left, right: pd.merge(left, right, on="Player", how="outer"), dfs)

                # These are common to all rows in the merged_df since added only in Summary
                if "Team" not in merged_df.columns:
                    merged_df["Team"] = team
                    merged_df["Opponent"] = opponent
                    merged_df["Home/Away"] = side

                final_rows.append(merged_df)

    except Exception as e:
        print(f"❌ Error with match {home_team} vs {away_team}: {e}")

    time.sleep(1)

driver.quit()

# === Combine all matches into one big DataFrame ===
if final_rows:
    all_matches_df = pd.concat(final_rows, ignore_index=True)

    # Drop Match URL if present (just in case)
    if "Match URL" in all_matches_df.columns:
        all_matches_df = all_matches_df.drop(columns=["Match URL"])

    # Drop duplicates for these fields, keeping only Summary__ versions
    target_suffixes = ["Nation", "Age", "Min", "#", "Pos"]
    columns_to_drop = [
        col for suffix in target_suffixes
        for col in all_matches_df.columns
        if col.endswith(f"__{suffix}") and not col.startswith("Summary__")
    ]
    all_matches_df = all_matches_df.drop(columns=columns_to_drop)

    # Drop Summary__ versions of metadata fields (keep plain ones)
    meta_columns_to_drop = ['Summary__Team', 'Summary__Opponent', 'Summary__Home/Away']
    all_matches_df = all_matches_df.drop(columns=[col for col in meta_columns_to_drop if col in all_matches_df.columns])

    # Clean Summary__Nation to keep only 3-letter country code
    if "Summary__Nation" in all_matches_df.columns:
        all_matches_df["Summary__Nation"] = all_matches_df["Summary__Nation"].str.extract(r'\b([A-Z]{3})\b')

    # Save cleaned output
    all_matches_df.to_csv(os.path.expanduser("~/Downloads/pl_merged_stats.csv"), index=False)
    print("✅ Saved cleaned merged stats to Downloads.")
else:
    print("⚠️ No data collected.")

🔄 Processing: Manchester Utd vs Fulham
🔄 Processing: Ipswich Town vs Liverpool
🔄 Processing: Newcastle Utd vs Southampton
🔄 Processing: Everton vs Brighton
🔄 Processing: Nott'ham Forest vs Bournemouth
🔄 Processing: Arsenal vs Wolves
🔄 Processing: West Ham vs Aston Villa
🔄 Processing: Brentford vs Crystal Palace
🔄 Processing: Chelsea vs Manchester City
🔄 Processing: Leicester City vs Tottenham
🔄 Processing: Aston Villa vs Arsenal
🔄 Processing: Crystal Palace vs West Ham
🔄 Processing: Fulham vs Leicester City
🔄 Processing: Manchester City vs Ipswich Town
🔄 Processing: Southampton vs Nott'ham Forest
🔄 Processing: Brighton vs Manchester Utd
🔄 Processing: Tottenham vs Everton
🔄 Processing: Wolves vs Chelsea
🔄 Processing: Bournemouth vs Newcastle Utd
🔄 Processing: Liverpool vs Brentford
🔄 Processing: Everton vs Bournemouth
🔄 Processing: West Ham vs Manchester City
🔄 Processing: Leicester City vs Aston Villa
🔄 Processing: Ipswich Town vs Fulham
🔄 Processing: Nott'ham Forest vs Wolves
🔄 Proces